Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras.datasets import cifar10
from tensorflow.keras import layers
from tensorflow.keras.models import Model

# Data Preparation

Load the dataset

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

Normalize data

In [3]:
X_train, X_test = X_train/255, X_test/255

Select Samples

# Apply Style Transfer

Load the Style Transfer Model

In [4]:
model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

Load Style Images

In [5]:
def load_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = img[tf.newaxis, :]
    return img

In [6]:
style_images = []
styles = ["Frida.jpeg","Michael-Jackson.jpeg","Starry-Night.jpeg", "mumya.jpg"]
for style in styles:
    style_images.append(load_image(f"../styles/{style}"))

Convert Images to TensorFlow Type

In [7]:
content_images = []

for i in range(20):
    content = tf.image.convert_image_dtype(X_train[i], tf.float32)
    content = content[tf.newaxis, :]
    content_images.append(content)

Generate Positive and Negative Samples

In [8]:
def generate_samples(content_images, style_images, style_transfer_model):
    negative_samples = []
    positive_samples = []
    # Generate positive samples
    for content in content_images:

        for style in style_images:
            stylized_image = style_transfer_model(tf.constant(content), tf.constant(style))[0]
            positive_samples.append((content, stylized_image))
    
    # Generate negative samples
    for content1 in content_images:
        for content2 in content_images:
            if  not (np.array_equal(content1, content2)):
                random_style = style_images[np.random.randint(0, len(style_images)-1)]
                stylized_image = style_transfer_model(tf.constant(content1), tf.constant(random_style))[0]
                negative_samples.append((content1, stylized_image))
    
    return positive_samples, negative_samples

# Constrastive Model

In [9]:
# Define the contrastive model architecture
def create_contrastive_model(input_shape_content, input_shape_style, embedding_dim):
    input_content = tf.keras.Input(shape=input_shape_content)
    input_style = tf.keras.Input(shape=input_shape_style)
    
    # Content branch
    x_content = layers.Conv2D(32, (3, 3), activation='relu')(input_content)
    x_content = layers.MaxPooling2D((2, 2))(x_content)
    x_content = layers.Conv2D(64, (3, 3), activation='relu')(x_content)
    x_content = layers.MaxPooling2D((2, 2))(x_content)
    x_content = layers.Flatten()(x_content)
    x_content = layers.Dense(embedding_dim, activation='relu')(x_content)
    
    # Style branch
    x_style = layers.Conv2D(32, (3, 3), activation='relu')(input_style)
    x_style = layers.MaxPooling2D((2, 2))(x_style)
    x_style = layers.Conv2D(64, (3, 3), activation='relu')(x_style)
    x_style = layers.MaxPooling2D((2, 2))(x_style)
    x_style = layers.Flatten()(x_style)
    x_style = layers.Dense(embedding_dim, activation='relu')(x_style)
    
    # Concatenate content and style embeddings
    concatenated = layers.Concatenate()([x_content, x_style])
    
    flattened = layers.Flatten()(concatenated)
    # Normalize the concatenated embeddings
    normalized = layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(concatenated)
    
    # Create the model with inputs and outputs
    model = tf.keras.Model(inputs=[input_content, input_style], outputs=normalized)
    return model


def contrastive_loss(y_true, y_pred, margin=1.0):
    square_pred = tf.square(1 - y_pred)
    margin_square = tf.square(tf.maximum(margin - y_pred, 0))
    return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)

Generate Samples

In [10]:
positive_samples, negative_samples = generate_samples(content_images, style_images, model)

Create Contrastive Model

In [11]:
contrastive_model = create_contrastive_model(input_shape_content=(32, 32, 3),
                                             input_shape_style=(32, 32, 3),
                                             embedding_dim=128)

In [12]:
positive_samples = np.array(positive_samples)
negative_samples = np.array(negative_samples)

In [13]:
print(f"p:{len(positive_samples)} | n: {len(negative_samples)}")

p:80 | n: 380


In [14]:
content_images_positive = positive_samples[:, 0]
stylized_images_positive = positive_samples[:, 1]
content_images_negative = negative_samples[:, 0]
stylized_images_negative = negative_samples[:, 1]
positive_labels = np.ones(len(positive_samples))
negative_labels = np.zeros(len(negative_samples))
train_images_content = np.concatenate((content_images_positive, content_images_negative))
train_images_style = np.concatenate((stylized_images_positive, stylized_images_negative))
train_labels_content = np.concatenate((positive_labels, negative_labels))

train_images_content = np.squeeze(train_images_content, axis=1)
train_images_style = np.squeeze(train_images_style, axis=1)

# Shuffle the data
indices = np.random.permutation(len(train_images_content))
train_images_content = train_images_content[indices]
train_images_style = train_images_style[indices]
train_labels_content = train_labels_content[indices]

# Select random samples for test data
num_test_samples = 50
test_indices = np.random.choice(len(train_images_content), size=num_test_samples, replace=False)
test_images_content = train_images_content[test_indices]
test_images_style = train_images_style[test_indices]
test_labels_content = train_labels_content[test_indices]

# Remove the selected test samples from the training set
train_images_content = np.delete(train_images_content, test_indices, axis=0)
train_images_style = np.delete(train_images_style, test_indices, axis=0)
train_labels_content = np.delete(train_labels_content, test_indices)

# Prepare the training data
train_data = ([train_images_content, train_images_style], train_labels_content)

# Prepare the test data
test_data = ([test_images_content, test_images_style], test_labels_content)

# Split the train_data into content and style inputs
train_content_images = train_data[0][0]
train_style_images = train_data[0][1]
train_labels = train_data[1]

test_content_images = test_data[0][0]
test_style_images = test_data[0][1]
test_labels = test_data[1]

# TEST

In [15]:
contrastive_model.compile(optimizer='adam', loss=contrastive_loss)
contrastive_model.fit([train_content_images, train_style_images], train_labels, epochs=10)

Epoch 1/10
13/13 [==============================] - 3s 57ms/step - loss: 0.8999
Epoch 2/10
13/13 [==============================] - 1s 64ms/step - loss: 0.8907
Epoch 3/10
13/13 [==============================] - 1s 60ms/step - loss: 0.8898
Epoch 4/10
13/13 [==============================] - 1s 58ms/step - loss: 0.8894
Epoch 5/10
13/13 [==============================] - 1s 48ms/step - loss: 0.8892
Epoch 6/10
13/13 [==============================] - 1s 52ms/step - loss: 0.8891
Epoch 7/10
13/13 [==============================] - 1s 61ms/step - loss: 0.8889
Epoch 8/10
13/13 [==============================] - 1s 51ms/step - loss: 0.8888
Epoch 9/10
13/13 [==============================] - 1s 53ms/step - loss: 0.8887
Epoch 10/10
13/13 [==============================] - 1s 53ms/step - loss: 0.8887


In [16]:
from keras.layers import Dense

In [ ]:
# Freeze the contrastive model
for layer in contrastive_model.layers:
    layer.trainable = False

# Remove the last layer (Lambda layer for l2 normalization)
contrastive_model = tf.keras.Model(contrastive_model.input, contrastive_model.layers[-2].output)

# Add a new classifier layer
output_layer = tf.keras.layers.Dense(10, activation='softmax')(contrastive_model.output)

# Define the new model for classification
classification_model = tf.keras.Model(contrastive_model.input, output_layer)

classification_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Train the classification model
classification_model.fit([X_train, X_train], y_train, epochs=10,validation_data=([X_test, X_test], y_test))


Epoch 1/10
1563/1563 [==============================] - 30s 19ms/step - loss: 2.1743 - accuracy: 0.1844 - val_loss: 2.1061 - val_accuracy: 0.2091
Epoch 2/10
 596/1563 [==========>...................] - ETA: 17s - loss: 2.1317 - accuracy: 0.2036

In [ ]:
# from keras.layers import Flatten
# from keras.utils import to_categorical
# # Assuming model_6 is your original model
# base_model = contrastive_model.layers[0]
# # Here I assumed that base_model is the part that deals with content image only

# # Now, add a new Flatten layer to your base model
# flattened = Flatten()(base_model.output)

# # Add a new classification layer to the flattened output
# output_layer = Dense(10, activation='softmax')(flattened)

# # Create a new model
# classification_model = Model(inputs=base_model.input, outputs=output_layer)
# y_train = to_categorical(y_train, num_classes=10)
# y_test = to_categorical(y_test, num_classes=10)
# # Now you can compile and train this model
# classification_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# classification_model.fit(X_train, y_train, epochs=10, validation_data=(X_test,y_test), batch_size=20)
